# Max value for unique values in column
- I found the translation between pandas and SQL somewhat non-intuitive for aggregation of data where I want to group by a specific column and keep ALL rows based on a condition (max/min) in another column
- In folowing example I want the maximum value in column_3 for unique values in column_2 while retaining value from column_1 
- Compare pandas vs SQL pattern

In [1]:
import pandas as pd

In [2]:
example = pd.DataFrame(dict(col_1=[1,2,3,4,5], col_2=['a', 'a', 'b', 'b', 'c'], col_3=[3,4,2,1,0]))
example

,col_1,col_2,col_3
0,1,a,3
1,2,a,4
2,3,b,2
3,4,b,1
4,5,c,0


### Pandas Pattern
- sort the dataframe by col_3 (descending)
- now drop duplicates in col_2 keeping the "first" value
- the "first" value will be the max because the dataframe was sorted in preceeding chained method

In [3]:
example.sort_values('col_3', ascending=False).drop_duplicates('col_2', keep='first')

,col_1,col_2,col_3
1,2,a,4
2,3,b,2
4,5,c,0


### SQL Pattern Oracle
- DB TESTED ON: Oracle 11g R2
- Note that temp_table is defined just to avoid having to create a real table
- Also note that aggregated table is stored using WITH syntax so that it can be separated from the query that orders it by col_3. Ordering by Col_3 is not a necessary step for the computation rather it is done to make the output appear the same as the pandas output. 

```SQL

WITH temp_table AS (
SELECT 1 AS col_1, 'a' AS col_2 ,3 AS col_3 FROM DUAL
UNION ALL
SELECT 2 AS col_1, 'a' AS col_2, 4 AS col_3 FROM DUAL
UNION ALL
SELECT 3 AS col_1, 'b' AS col_2, 2 AS col_3 FROM DUAL
UNION ALL
SELECT 4 AS col_1, 'b' AS col_2, 1 AS col_3 FROM DUAL
UNION ALL
select 5 AS col_1, 'c' AS col_1, 0 AS col_3 FROM DUAL

),

aggregated AS (SELECT DISTINCT
FIRST_VALUE(col_1) OVER (PARTITION BY col_2 ORDER BY col_3 DESC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS col_1,
FIRST_VALUE(col_2) OVER (PARTITION BY col_2 ORDER BY col_3 DESC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS col_2,
FIRST_VALUE(col_3) OVER (PARTITION BY col_2 ORDER BY col_3 DESC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS col_3
FROM temp_table)

SELECT * 
FROM aggregated
ORDER BY col_3 DESC;
```

#### OUTPUT
```
| COL_1 | COL_2 | COL_3 |
|-------|-------|-------|
|     2 |     a |     4 |
|     3 |     b |     2 |
|     5 |     c |     0 |
```